In [2]:
import pandas as pd

In [3]:
# read text files and chunk them into sentences with some overlapping
import os
import re

def chunk_text(file_path, overlap=1):
    with open(file_path, 'r') as file:
        text = file.read()
    chunk_size = 100
    step_size = int(chunk_size * (1 - overlap / 10))
    chunks = [text[i:i + chunk_size] for i in range(0, len(text), step_size)]
    return chunks

df = None

for file in os.listdir('文学'):
    if file.endswith('.txt'):
        file_path = os.path.join('文学', file)
        book_name = file.split('-')[0]
        author = file.split('-')[1]
        year = file.split('-')[2].split('.')[0]
        chunks = chunk_text(file_path, overlap=1)
        df_book = pd.DataFrame(
            {'text': chunks,
             'book': [book_name] * len(chunks),
             'author': [author] * len(chunks),
             'year': [year] * len(chunks)}
        )
        df_book['text'] = df_book['text'].str.replace(r'\n', ' ', regex=True).str.strip()

        df = pd.concat([df, df_book], ignore_index=True) if df is not None else df_book

df

,text,book,author,year
0,世紀末的華麗 這是台灣獨有的城市天際線，米亞常常站在她的九樓陽臺上觀測天象。依...,世紀末的華麗,朱天文,1990
1,伸到日出日落處。我們需要輕質化建築，米亞的情人老段說。老段用輕質沖孔鐵皮建材來解決別墅開天窗...,世紀末的華麗,朱天文,1990
2,米亞是一位相信嗅覺，依賴嗅覺記憶活著的人。安息香使她回到那場八九年春裝秀中，淹沒在一片雪紡、...,世紀末的華麗,朱天文,1990
3,巾，搭配前個世紀末展露於維也納建築繪畫中的裝飾風，其間翹楚克林姆，綴滿亮箔珠繡的裝飾風。 ...,世紀末的華麗,朱天文,1990
4,，但她確信只要被她遇見一定逃不掉，然後那一種紫色負荷的所有東西霎時都會重現。不過比起嗅覺，顏...,世紀末的華麗,朱天文,1990
...,...,...,...,...
11154,隨時隨地都可能、容易受到各種意外巧合的襲擊，並因此遭遇死亡。他們像原始人似的必須天天面對充滿...,預知死亡紀事,朱天心,1992
11155,誕不經、笑破肚腸、而他們所認為的神祕徵兆。 曠野之子（太陽晒熟的美果，月亮養成的寶貝），我...,預知死亡紀事,朱天心,1992
11156,曠野之子，他死得太早，假若活到我這年紀，他也許要收回他的教義—— 我們的老靈魂們，我無法再...,預知死亡紀事,朱天心,1992
11157,的歌聲： 或許夜行者， 把這月暈叫作氣象， 但是我們精靈看法不同， 只有我們持有正確...,預知死亡紀事,朱天心,1992


In [4]:
import os
from IPython.display import display, HTML
import ipywidgets as widgets

def highlight_keywords_toolkit(df, iloc):
    """
    Creates an interactive toolkit for highlighting keywords in a specific text from the dataframe.

    Parameters:
    - df: pandas DataFrame containing the text data.
    - iloc: int, the row index of the text to highlight.
    """
    # Validate iloc index
    if iloc < 0 or iloc >= len(df):
        print("Invalid iloc index.")
        return

    # Get the text for the specified iloc
    text = df.iloc[iloc]['text']
    book = df.iloc[iloc]['book']
    author = df.iloc[iloc]['author']
    year = df.iloc[iloc]['year']

    # Create widgets
    text_area = widgets.Textarea(value=text, description='Text:', layout=widgets.Layout(width='100%', height='200px'))
    keyword_input = widgets.Text(description='Keywords:', placeholder='Enter keywords separated by commas')
    save_button = widgets.Button(description='Save Keywords')
    output_label = widgets.Label(value='')

    # Function to handle keyword highlighting and saving
    def save_keywords(_):
        keywords = [kw.strip() for kw in keyword_input.value.split(',') if kw.strip()]
        if not keywords:
            output_label.value = "No keywords entered."
            return

        # Highlight keywords in the text
        highlighted_text = text
        for keyword in keywords:
            highlighted_text = re.sub(f"({re.escape(keyword)})", r'<mark>\1</mark>', highlighted_text, flags=re.IGNORECASE)

        # Save keywords to a file
        output_file = f"./labeled_keywords/{iloc}.txt"
        with open(output_file, 'w', encoding='utf-8') as f:
            f.write(', '.join(keywords))
        
        # Display the highlighted text and confirmation
        display(HTML(f"<div style='font-family: Arial; line-height: 1.6;'>{highlighted_text}</div>"))
        output_label.value = f"Keywords saved to {output_file}"

    # Attach the save function to the button
    save_button.on_click(save_keywords)

    # Display the interface
    display(widgets.VBox([text_area, keyword_input, save_button, output_label]))


random_index = df.index.to_list()
import random
random.shuffle(random_index)
for idx in random_index[:10]:
    highlight_keywords_toolkit(df, idx)
